# Predict attrition based on various factors
- Individual Project by Vijay Zarapala

## Objective
* Predict the impact of factors like BusinessTravel, Education, EnvironmentSatisfaction, Gender, JobSatisfaction, MaritalStatus in contribution to attrition.

* Used random forrest
  * Trained on a portion of the dataset
  * Tested the trained model against the remainder of the dataset
    * Accuracy can be determined because the dataset is labeled (i.e., this uses supervised learning)

## Download dataset
  * Dataset: https://www.kaggle.com/pavansubhasht/ibm-hr-analytics-attrition-dataset/data

In [4]:
%sh
mkdir -p kaggle_project
curl 'https://pastebin.com/raw/vX7WErKF' > kaggle_project/hr.data 
#curl 'https://drive.google.com/open?id=1RQxs6F3budRrMBOXNCA68OcQyhjVbPWv' > kaggle_project/hr.data
#curl 'https://www.dropbox.com/s/4bje2frj2seb7wb/WA_Fn-UseC_-HR-Employee-Attrition.csv?dl=0' > kaggle_project/hr.data
ls kaggle_project

In [5]:
#read the hr.data file as CSV
df_data = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .option('inferSchema', 'true')\
  .load("file:/databricks/driver/kaggle_project/hr.data")
df_data.show(2)

In [6]:
df_data.printSchema()

##Select only the relevant columns from the dataset

In [8]:
df = df_data.selectExpr('Age as Age','Attrition as Attrn', 'BusinessTravel as Travel', 'Department as Dept', 'Education as Edu', 'EnvironmentSatisfaction as Environ', 'Gender as Sex','HourlyRate as HR','JobLevel as JL','JobSatisfaction as JobS','MaritalStatus as MS','NumCompaniesWorked as TotCompanies','OverTime as OT','PercentSalaryHike as Hike','PerformanceRating as Rating', 'WorkLifeBalance as WLB','YearsSinceLastPromotion as LastPromotion')

df.show(30)

#df = df_data.selectExpr('Age','Attrition', 'BusinessTravel', 'Department', 'Education', 'EducationField', 'EnvironmentSatisfaction', 'Gender','HourlyRate','JobLevel','JobSatisfaction','MaritalStatus','NumCompaniesWorked', 'OverTime', 'PercentSalaryHike','PerformanceRating', 'RelationshipSatisfaction', 'TotalWorkingYears', 'WorkLifeBalance', 'YearsAtCompany', 'YearsSinceLastPromotion')
#df = df_data
#df = df_data.selectExpr('Age as Age','Attrition as Attrition', 'BusinessTravel as BusinessTravel', '_c5 as marital_status', '_c6 as occupation', '_c8 as race', '_c9 as sex', '_c12 as hoursPerWeek', '_c13 as native_country','_c14 as Salary')

## Data explored and explained

In [10]:
df.printSchema()

In [11]:
df.show(30)

In [12]:
df.registerTempTable('hrTbl')

In [13]:
%sql

select 
count(distinct Age) as AgeC, 
count(distinct Attrn) as AttrnC, 
count(distinct Travel) as TravelC, 
count(distinct Dept) as DeptC, 
count(distinct Edu) as  EduC, 
count(distinct Environ) as   EnvironC, 
count(distinct Sex) as   SexC, 
count(distinct HR) as   HRc, 
count(distinct JL) as   JLC,  
count(distinct JobS) as JobSC, 
count(distinct MS) as   MSC, 
count(distinct TotCompanies) as   TotCompaniesC, 
count(distinct OT) as OTC, 
count(distinct Hike) as HikeC, 
count(distinct Rating) as RatingC, 
count(distinct WLB) as WLBC, 
count(distinct LastPromotion) as LastPromotionC  from hrTbl;



In [14]:
df.select('Attrn').distinct().show()

In [15]:
df.select('Attrn').groupBy('Attrn').count().show()

In [16]:
df.count()

## Data Cleaning

In [18]:
# NOT REQUIRED AS THE DATA IS CLEAN

## Data transformation

In [20]:
splitted_data = df.randomSplit([0.9, 0.05, 0.05], 7) # 90-training, 5-testing, 5-prediction - gives 91% accuracy with seed
#splitted_data = df.randomSplit([0.8, 0.1, 0.1]) # 80-training, 10-testing, 10-prediction - gives 88% accuracy without seed
train_data = splitted_data[0]
test_data = splitted_data[1]
predict_data = splitted_data[2]

print "Number of training records: " + str(train_data.count())
print "Number of testing records : " + str(test_data.count())
print "Number of prediction records : " + str(predict_data.count())

## Data modeling

In [22]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline, Model

In [23]:
#Check the schema once again to know the names of the columns
df.printSchema()

In [24]:
AgeTypes = int(df.select('Age').distinct().count())
AttritionTypes = int(df.select('Attrn').distinct().count())
TravelTypes = int(df.select('Travel').distinct().count())
DeptTypes = int(df.select('Dept').distinct().count())
EduTypes = int(df.select('Edu').distinct().count())
EnvironTypes = int(df.select('Environ').distinct().count())
SexTypes = int(df.select('Sex').distinct().count())
HRTypes = int(df.select('HR').distinct().count()) 
JLTypes = int(df.select('JL').distinct().count())
JobSTypes = int(df.select('JobS').distinct().count())
MSTypes = int(df.select('MS').distinct().count())
TotCompaniesTypes = int(df.select('TotCompanies').distinct().count())
OTTypes = int(df.select('OT').distinct().count())
HikeTypes = int(df.select('Hike').distinct().count())
RatingTypes = int(df.select('Rating').distinct().count())
WLBTypes = int(df.select('WLB').distinct().count())
LastPromotionTypes = int(df.select('LastPromotion').distinct().count())

print [
  ['AgeTypes',AgeTypes ],
  ['AttritionTypes',AttritionTypes],
  ['TravelTypes', TravelTypes ],
  ['DeptTypes', DeptTypes ],
  ['EduTypes', EduTypes ],
  ['EnvironTypes', EnvironTypes],
  ['SexTypes', SexTypes ],
  ['HRTypes', HRTypes ],
  ['JLTypes', JLTypes ],
  ['JobSTypes', JobSTypes],
  ['MSTypes', MSTypes ],
  ['TotCompaniesTypes', TotCompaniesTypes ],
  ['OTTypes', OTTypes ],
  ['HikeTypes', HikeTypes],
  ['RatingTypes', RatingTypes],
  ['WLBTypes', WLBTypes ],
  ['LastPromotionTypes', LastPromotionTypes]]

resultDF = sqlContext.createDataFrame([
  ['AgeTypes',AgeTypes ],
  ['AttritionTypes',AttritionTypes],
  ['TravelTypes', TravelTypes ],
  ['DeptTypes', DeptTypes ],
  ['EduTypes', EduTypes ],
  ['EnvironTypes', EnvironTypes],
  ['SexTypes', SexTypes ],
  ['HRTypes', HRTypes ],
  ['JLTypes', JLTypes ],
  ['JobSTypes', JobSTypes],
  ['MSTypes', MSTypes ],
  ['TotCompaniesTypes', TotCompaniesTypes ],
  ['OTTypes', OTTypes ],
  ['HikeTypes', HikeTypes],
  ['RatingTypes', RatingTypes],
  ['WLBTypes', WLBTypes ],
  ['LastPromotionTypes', LastPromotionTypes]], 
  ['metric', 'value'])
display(resultDF)

## Observation:
 * Thus we see that there are <b>large number of distinct values</b> for the columns <b>Age</b>, <b>HourlyRate</b>, <b>Salary Hike</b> and <b>Last Promotion</b>.
 * We will leave out these features as it will affect the accuracy of our model.
 * Thus we will consider the factors such as Business Travel, Education, EnvironmentSatisfaction, Gender, JobSatisfaction, MaritalStatus and see how these impact the attrition.

In [26]:
stringIndexer_label = StringIndexer(inputCol="Attrn", outputCol="label").fit(df)
# Note the above outputCol is label (the predicted column). Here we predict from the attributes below.
stringIndexer_BT = StringIndexer(inputCol="Travel", outputCol="BT_IX")
stringIndexer_EDU = StringIndexer(inputCol="Edu", outputCol="EDU_IX")
stringIndexer_ES = StringIndexer(inputCol="Environ", outputCol="ES_IX")
stringIndexer_SEX = StringIndexer(inputCol="Sex", outputCol="SEX_IX")
stringIndexer_JS = StringIndexer(inputCol="JobS", outputCol="JS_IX")
stringIndexer_MS = StringIndexer(inputCol="MS", outputCol="MS_IX")

In [27]:
# Select the input columns for the model (and put them into one features column)
vectorAssembler_features = VectorAssembler(inputCols=["BT_IX", "EDU_IX", "ES_IX", "SEX_IX", "JS_IX", "MS_IX"], outputCol="features")

In [28]:
# The model
rf = RandomForestClassifier(labelCol="label", featuresCol="features")

In [29]:
# Columns for the output
# Convert from indexed labels (added above) back to original labels.
# https://spark.apache.org/docs/2.0.2/api/python/pyspark.ml.html#pyspark.ml.feature.IndexToString
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=stringIndexer_label.labels)

In [30]:
# The ML pipeline
pipeline_rf = Pipeline(stages=[stringIndexer_label, stringIndexer_BT, stringIndexer_EDU, stringIndexer_ES, stringIndexer_SEX, stringIndexer_JS, stringIndexer_MS,  vectorAssembler_features, rf, labelConverter])

In [31]:
# Model training
model_rf = pipeline_rf.fit(train_data)

## Model evaluation

In [33]:
# Model quality
predictions = model_rf.transform(test_data)
evaluatorRF = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluatorRF.evaluate(predictions)
print("Accuracy = %g" % accuracy)
print("Test Error = %g" % (1.0 - accuracy))

In [34]:
display(predictions)

In [35]:
#Number of predictions predicting
predictions.select('prediction').groupBy('prediction').count().show()

In [36]:
correct = predictions.where("(label = prediction)").count()
incorrect = predictions.where("(label != prediction)").count()

resultDF = sqlContext.createDataFrame([['correct', correct], ['incorrect', incorrect]], ['metric', 'value'])
display(resultDF)